## Is there a relation between nucleic acid conformation and stacking with the protein?
- Which features of the nucleic acid conformation?
- How do we define stacking?

In [1]:
import numpy as np

**Select PDB**

In [2]:
pdb_code = "1B7F"
na_chain = "P"
protein_chain = "A"
na_resid = 5
protein_resid = 256

**Visualize PDB**

In [3]:
import nglview
widget = nglview.NGLWidget()
widget.add_component("rcsb://" + pdb_code)
selection='({0} and :{1}) or ({2} and :{3})'.format(na_resid, na_chain, protein_resid, protein_chain)
widget.add_representation('ball+stick', selection=selection, color='blue')
widget.center(selection)
widget

NGLWidget()

**Download PDB**

In [4]:
import urllib
pdb_data = urllib.request.urlopen("https://files.rcsb.org/download/{}.pdb".format(pdb_code)).read().decode()

**Execute X3DNA-DSSR**

In [5]:
import tempfile
import subprocess
import json
import os
with tempfile.NamedTemporaryFile(mode="w") as tempfile_pdb:
    tempfile_pdb.write(pdb_data)
    with tempfile.NamedTemporaryFile(mode="r") as tempfile_result:
        cwd = os.getcwd()
        try:
            os.chdir(tempfile.gettempdir())
            subp = subprocess.run(
                "x3dna-dssr -i={0} --json -o={1}".format(tempfile_pdb.name, tempfile_result.name),
                shell=True,
                stderr=subprocess.PIPE
            )
        finally:
            os.chdir(cwd)
        print(subp.stderr.decode())
        assert subp.returncode == 0        
        x3dna_analysis_data = tempfile_result.read()        
        x3dna_analysis = json.loads(x3dna_analysis_data)

[i] JSON output should end with the .json extension

Processing file '/tmp/tmp320kf1u_'
    total number of base pairs: 2
    total number of atom-base capping interactions: 2
    total number of splayed-apart dinucleotides: 12
    total number of 	consolidated into units: 8
    total number of non-loop single-stranded segments: 2

Time used: 00:00:00:00



**Select chain and show X3DNA results**

In [6]:
x3dna_nucleotides = [nt for nt in x3dna_analysis["nts"] if nt["chain_name"] == na_chain]
x3dna_nucleotides[0]

{'index': 1,
 'index_chain': 1,
 'chain_name': 'P',
 'nt_resnum': 1,
 'nt_name': 'G',
 'nt_code': 'G',
 'nt_id': 'P.G1',
 'dbn': '.',
 'alpha': None,
 'beta': -92.219,
 'gamma': -161.658,
 'delta': 127.686,
 'epsilon': -167.738,
 'zeta': 70.091,
 'epsilon_zeta': 122.17,
 'bb_type': '..',
 'chi': 37.927,
 'baseSugar_conf': 'syn',
 'form': '.',
 'ssZp': -2.512,
 'Dp': 2.834,
 'splay_angle': 104.33,
 'splay_distance': 15.063,
 'splay_ratio': 0.79,
 'eta': None,
 'theta': -37.5,
 'eta_prime': None,
 'theta_prime': -15.289,
 'eta_base': None,
 'theta_base': -8.995,
 'v0': -39.548,
 'v1': 41.741,
 'v2': -27.038,
 'v3': 4.29,
 'v4': 21.27,
 'amplitude': 41.84,
 'phase_angle': 130.258,
 'puckering': "C1'-exo",
 'sugar_class': "~C2'-endo",
 'bin': 'inc',
 'cluster': '__',
 'suiteness': 0.0,
 'filter_rmsd': 0.014,
 'frame': {'rsmd': 0.014,
  'origin': [40.637, 51.477, 117.66],
  'x_axis': [0.41, -0.861, 0.301],
  'y_axis': [0.715, 0.097, -0.693],
  'z_axis': [0.567, 0.499, 0.655],
  'quaternion'

In [7]:
import pandas as pd
df_x3dna = pd.DataFrame(x3dna_nucleotides)
df_x3dna

,index,index_chain,chain_name,nt_resnum,nt_name,nt_code,nt_id,dbn,alpha,beta,...,v4,amplitude,phase_angle,puckering,sugar_class,bin,cluster,suiteness,filter_rmsd,frame
0,1,1,P,1,G,G,P.G1,.,NaN,-92.219,...,21.270,41.840,130.258,C1'-exo,~C2'-endo,inc,__,0.000,0.014,"{'rsmd': 0.014, 'origin': [40.637, 51.477, 117..."
1,2,2,P,2,U,U,P.U2,.,-108.087,-177.928,...,-6.128,33.763,171.949,C2'-endo,~C2'-endo,22p,!!,0.000,0.100,"{'rsmd': 0.022, 'origin': [42.908, 63.87, 109...."
2,3,3,P,3,U,U,P.U3,.,-137.293,-132.284,...,20.740,40.514,130.075,C1'-exo,~C2'-endo,22p,!!,0.000,0.102,"{'rsmd': 0.02, 'origin': [32.545, 49.313, 107...."
3,4,4,P,4,G,G,P.G4,.,-75.588,-171.481,...,2.913,34.945,156.181,C2'-endo,~C2'-endo,22p,2[,0.176,0.008,"{'rsmd': 0.008, 'origin': [34.019, 62.087, 102..."
4,5,5,P,5,U,U,P.U5,.,-92.225,168.414,...,2.480,28.247,156.328,C2'-endo,~C2'-endo,22p,0b,0.404,0.097,"{'rsmd': 0.017, 'origin': [25.464, 59.579, 96...."
5,6,6,P,6,U,U,P.U6,.,113.213,-149.409,...,17.053,27.209,122.934,C1'-exo,~C2'-endo,trig,!!,0.000,0.098,"{'rsmd': 0.015, 'origin': [22.219, 51.733, 86...."
6,7,7,P,7,U,U,P.U7,.,-46.035,172.227,...,-1.802,27.027,338.310,C2'-exo,~C3'-endo,trig,!!,0.000,0.099,"{'rsmd': 0.018, 'origin': [35.168, 62.432, 86...."
7,8,8,P,8,U,U,P.U8,.,-43.995,161.969,...,26.189,31.277,37.851,C4'-exo,~C3'-endo,33p,1a,0.113,0.099,"{'rsmd': 0.017, 'origin': [32.138, 63.711, 84...."
8,9,9,P,9,U,U,P.U9,.,68.875,159.521,...,3.121,25.674,154.769,C2'-endo,~C2'-endo,32p,7p,0.309,0.100,"{'rsmd': 0.013, 'origin': [16.339, 59.62, 91.8..."
9,10,10,P,10,U,U,P.U10,.,-64.981,164.796,...,-7.922,33.764,174.896,C2'-endo,~C2'-endo,22p,4b,0.458,0.095,"{'rsmd': 0.015, 'origin': [13.775, 67.362, 89...."


**Parse PDB into structured Numpy array using parse_pdb.py**

In [8]:
import parse_pdb

In [9]:
parsed_pdb = parse_pdb.parse_pdb(pdb_data)
parsed_pdb[:2]

array([(1, b' ', b'OP3', b' ', b'  G', b'P', 1, b' ', 1, 43.063, 59.607, 115.478, 1., 68.76, b'    ', b'O'),
       (1, b' ', b'P', b' ', b'  G', b'P', 2, b' ', 1, 42.131, 58.379, 115.046, 1., 68.5 , b'    ', b'P')],
      dtype={'names':['model','hetero','name','altloc','resname','chain','index','icode','resid','x','y','z','occupancy','bfactor','segid','element'], 'formats':['<u2','S1','S4','S1','S3','S1','<u4','S1','<u2','<f4','<f4','<f4','<f4','<f4','S4','S2'], 'offsets':[0,2,3,7,8,11,12,16,18,20,24,28,32,36,40,44], 'itemsize':48, 'aligned':True})

In [10]:
parse_pdb.print_atom(parsed_pdb[:2])

[{'model': 1,
  'hetero': b' ',
  'name': b'OP3',
  'altloc': b' ',
  'resname': b'  G',
  'chain': b'P',
  'index': 1,
  'icode': b' ',
  'resid': 1,
  'x': 43.063,
  'y': 59.607,
  'z': 115.478,
  'occupancy': 1.0,
  'bfactor': 68.76,
  'segid': b'    ',
  'element': b'O'},
 {'model': 1,
  'hetero': b' ',
  'name': b'P',
  'altloc': b' ',
  'resname': b'  G',
  'chain': b'P',
  'index': 2,
  'icode': b' ',
  'resid': 1,
  'x': 42.131,
  'y': 58.379,
  'z': 115.046,
  'occupancy': 1.0,
  'bfactor': 68.5,
  'segid': b'    ',
  'element': b'P'}]

In [11]:
df_pdb = pd.DataFrame(parsed_pdb)
# Convert bytes to strings
for col, dtype in df_pdb.dtypes.items():
    if dtype == np.object:  # Only process byte object columns.
        df_pdb[col] = df_pdb[col].apply(lambda x: x.decode("utf-8"))
df_pdb[:10]

,model,hetero,name,altloc,resname,chain,index,icode,resid,x,y,z,occupancy,bfactor,segid,element
0,1,,OP3,,G,P,1,,1,43.063000,59.606998,115.477997,1.0,68.760002,,O
1,1,,P,,G,P,2,,1,42.131001,58.379002,115.045998,1.0,68.500000,,P
2,1,,OP1,,G,P,3,,1,40.734001,58.716999,115.407997,1.0,72.000000,,O
3,1,,OP2,,G,P,4,,1,42.750000,57.137001,115.561996,1.0,67.459999,,O
4,1,,O5',,G,P,5,,1,42.236000,58.367001,113.435997,1.0,65.599998,,O
5,1,,C5',,G,P,6,,1,41.796001,57.227001,112.693001,1.0,63.330002,,C
6,1,,C4',,G,P,7,,1,42.939999,56.250000,112.446999,1.0,61.330002,,C
7,1,,O4',,G,P,8,,1,43.192001,55.431000,113.620003,1.0,65.720001,,O
8,1,,C3',,G,P,9,,1,42.609001,55.277000,111.293999,1.0,59.939999,,C
9,1,,O3',,G,P,10,,1,43.695000,55.313000,110.344002,1.0,51.830002,,O


**Select protein and nucleic acid chain from parsed PDB**

In [12]:
protein_atoms = parsed_pdb[parsed_pdb["chain"]==protein_chain.encode()]
na_atoms = parsed_pdb[parsed_pdb["chain"]==na_chain.encode()]
len(protein_atoms), len(na_atoms)

(1365, 254)

**Define code to calculate stacking properties**

In [64]:
from sympy import Plane, Point3D
from math import acos, sqrt

def calculate_stacking_properties(protein_atoms, protein_resid, na_atoms, na_resid):
    import scipy.spatial.distance
    res_protein = protein_atoms[protein_atoms["resid"]==protein_resid]    
    assert len(res_protein)
    aa = res_protein[0]["resname"].decode().strip()
    res_na = na_atoms[na_atoms["resid"]==na_resid]
    assert len(res_na)
    nuc = res_na[0]["resname"].decode().strip()[-1] # one-letter
    coor_res_protein = np.stack((res_protein["x"], res_protein["y"], res_protein["z"])).T
    coor_res_na = np.stack((res_na["x"], res_na["y"], res_na["z"])).T
    
    result = {}
    dist = scipy.spatial.distance.cdist(coor_res_protein, coor_res_na)
    result["closest_distance"] = dist.min()
    
    if result["closest_distance"] > 6:
        result["stacking_angle"] = 0
        result["min_stacking_distance"] = 10
        return result

    sidechains = {
        "PHE": ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
        "TRP": ['CD', 'CZ2', 'CZ2', 'CE2', 'CE3', 'CH'],
        "TYR": ['CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ'],
        "HIS": ['CG', 'ND1', 'CD2', 'CE1', 'NE2'],
        "GLN": ['CD', 'OE1', 'NE2'],
        "ASP": ['OD1', 'OD2', 'CG'],
        "ARG": ['CZ', 'NH1', 'NH2'],
    }
    sidechain_mask = np.isin(res_protein["name"], [name.encode() for name in sidechains[aa]])
    bases = {
        "U": ['C2', 'C4', 'C5', 'C6', 'N1', 'N3'],
        "C": ['C2', 'C4', 'C5', 'C6', 'N1', 'N3'],
        "G": ['C2', 'C4', 'C5', 'C6', 'N1', 'N3', "N7", "C8","N9"],
        "A": ['C2', 'C4', 'C5', 'C6', 'N1', 'N3', "N7", "C8","N9"],
    }
    base_mask = np.isin(res_na["name"], [name.encode() for name in bases[nuc]])
    stacking_dist = dist[sidechain_mask][:,base_mask]
    result["min_stacking_distance"] = stacking_dist.min()
    #result["std_stacking_dist"] = stacking_dist.std()
    
    # 10.1261/rna.054924.115 (Suppl S8,S9) define stacking
    # by angle and min heavy-atom dist between the 
    # cycles (F, Y, W, H) or cycles-triangles (Q, D, R):
    #   min_dist in [2.7-4.3A], 
    #   angle in [0 - 20] for parall stacking
    #   angle in [70 - 90] for T-shape stacking.
    cycle_na = coor_res_na[base_mask]
    cycle_protein = coor_res_protein[sidechain_mask]
    
    plane_na = Plane(Point3D(cycle_na[0]), Point3D(cycle_na[1]), Point3D(cycle_na[2]))
    plane_protein = Plane(Point3D(cycle_protein[0]), Point3D(cycle_protein[1]), Point3D(cycle_protein[2]))
    
    rad = plane_na.angle_between(plane_protein).evalf()
    deg = round(rad*180/3.14159)
    d = min(deg, 180-deg)
    result["stacking_angle"] = d
    
    return result

In [65]:
stacking_properties = calculate_stacking_properties(protein_atoms, protein_resid, na_atoms, na_resid)
stacking_properties

{'closest_distance': 3.592381988189429,
 'min_stacking_distance': 3.592381988189429,
 'stacking_angle': 5}

**Define code to integrate all properties**
- Stacking properties are computed using the code above
- A list of other properties is extracted from the X3DNA analysis

In [58]:
def calculate_all_properties(protein_atoms, protein_resid, na_atoms, na_resid, x3dna_nucleotides):
    stacking_properties = calculate_stacking_properties(protein_atoms, protein_resid, na_atoms, na_resid)
    x3dna_nucl = [nucl for nucl in x3dna_nucleotides if nucl["nt_resnum"] == na_resid]
    assert len(x3dna_nucl) == 1
    nucl_props = ["gamma", "delta", "chi"]
    result = {}
    for prop in nucl_props:
        result[prop] = x3dna_nucl[0][prop]
    result.update(stacking_properties)
    return result

In [66]:
all_properties = calculate_all_properties(protein_atoms, protein_resid, na_atoms, na_resid, x3dna_nucleotides)
all_properties

{'gamma': 74.872,
 'delta': 138.48,
 'chi': -124.134,
 'closest_distance': 3.592381988189429,
 'min_stacking_distance': 3.592381988189429,
 'stacking_angle': 5}

**Calculate properties for all residue-nucleotide pair**

Instead of using the pre-selected residue and nucleotide, iterate over all

In [67]:
all_protein_resids = np.unique(protein_atoms["resid"])
all_na_resids = np.unique(na_atoms["resid"])

In [68]:
stackings = []
for curr_na_resid in all_na_resids:    
    for curr_protein_resid in all_protein_resids:
        try:
            properties = calculate_all_properties(
                protein_atoms, curr_protein_resid, 
                na_atoms, curr_na_resid, 
                x3dna_nucleotides
            )
        except (KeyError, AssertionError):
            continue
        properties["na_resid"] = curr_na_resid
        properties["protein_resid"] = curr_protein_resid
        stackings.append(properties)

In [69]:
df_stackings = pd.DataFrame(stackings)
df_stackings

,gamma,delta,chi,closest_distance,stacking_angle,min_stacking_distance,na_resid,protein_resid
0,-161.658,127.686,37.927,42.131576,0,10.0,1,123
1,-161.658,127.686,37.927,45.179849,0,10.0,1,124
2,-161.658,127.686,37.927,45.683074,0,10.0,1,125
3,-161.658,127.686,37.927,42.065976,0,10.0,1,126
4,-161.658,127.686,37.927,39.878556,0,10.0,1,127
...,...,...,...,...,...,...,...,...
2360,-89.086,129.720,86.807,16.574007,0,10.0,12,321
2361,-89.086,129.720,86.807,25.341204,0,10.0,12,322
2362,-89.086,129.720,86.807,22.658904,0,10.0,12,323
2363,-89.086,129.720,86.807,22.041585,0,10.0,12,324


In [71]:
df_stackings[(df_stackings.min_stacking_distance < 4.5) & (df_stackings.stacking_angle < 20)]

,gamma,delta,chi,closest_distance,stacking_angle,min_stacking_distance,na_resid,protein_resid
693,47.639,144.416,-123.797,2.957644,16,3.391428,4,214
929,74.872,138.480,-124.134,3.592382,5,3.592382,5,256
1004,73.571,118.594,-115.632,3.414097,16,3.414097,6,131
1066,73.571,118.594,-115.632,2.955630,12,3.455867,6,195
2013,-43.493,95.532,-124.546,3.277439,15,3.277439,11,170


**Plot a nucleotide conformation property versus a stacking property**

In [ ]:
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.scatter(
    [stacking["chi"] for stacking in stackings],
    [stacking["closest_distance"] for stacking in stackings],
)
ax.set_xlabel('Chi')
ax.set_ylabel('Closest distance')
plt.show()